# **Import libraries**

In [ ]:
import csv

In [ ]:
my_data = [] #우리의 전처리된 데이터가 여기에 담길 것

# **Personalized functions** 

In [ ]:
def hasNumber(stringVal) :
  return any(elem.isdigit() for elem in stringVal)

In [ ]:
def getDigit(stringVal) :
  onlyDigit = ''
  for i in stringVal :
    if (i.isdigit()) :
      onlyDigit += i
  return onlyDigit

# **Read data from csv and pre-process data**

In [ ]:
f = open('mr-boston-flattened.csv','r')
rdr = csv.reader(f)

line_count = 0
for line in rdr :
  if (line_count == 0) :
    line_count += 1
    pass
  else :
    #변수 초기화
    cocktail = [] #우리 데이터 한 row
    cocktail_ingredient_measure = [] #칵테일 성분 정보(딕셔너리의 리스트로 구성 [{성분1 : 용량}, {성분2 : 용량}, ... ])
    measurement_list = [] 
    ingred_idx = 3 #용량에 mapping 되는 ingredient idx
    new_measurement_list = [] #변경한 값 담는 배열
    
    for i in range(2,14,2) : #각 row에 measurement 뽑아서 list에 넣기
      measurement_list.append(line[i])

    for j in measurement_list : #measure
      if (hasNumber(j) == False) : #숫자가 하나도 없고 전부 문자열인 경우
        new_measurement_list.append(0)
      elif (j == '') : #빈 값인 경우 읽기 때문에 0으로 변경해줌
        new_measurement_list.append(0)

      else : #'1 1/2 oz' 형식으로 되어있음 따라서 공백으로 자르고 처리
        slice_str = []
        slice_str = j.split()
        total_val = 0
        
        for n in slice_str : #잘라진 '1' '1/2' 'oz' 처리
          if (n.isalpha()) :
            pass
          else :
            if (n.find('/') == -1) : #그냥 숫자인 경우
              num_tmp = getDigit(n)

              if (num_tmp == '') :
                num_tmp = 0

              total_val += int(num_tmp)
            else : # 1/2 인 경우
              idx = n.find('/')
              divide_val = int(n[:idx]) / int(n[idx+1:])
              total_val += divide_val
        new_measurement_list.append(total_val)
        

### 여기까지 오면 new_measurement_list에는 숫자만 들어있음
    total_measurement_value = sum(new_measurement_list)

    for j in new_measurement_list :
      if (line[ingred_idx] != '') :

        ing_measure_pair = {line[ingred_idx] : j}
        cocktail_ingredient_measure.append(ing_measure_pair)
      
      ingred_idx += 2

    #칵테일 이름
    cocktail.append(line[0])
    #칵테일 라벨
    cocktail.append(line[1])
    #성분 전체 용량
    cocktail.append(total_measurement_value)
    #각 성분별 성분명, 용량 딕셔너리
    cocktail.append(cocktail_ingredient_measure)
    my_data.append(cocktail)

f.close()